<a href="https://colab.research.google.com/github/FatemeZamanian/DeepLearning/blob/main/PersianRecognitionBot/persianRecognitionBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyTelegramBotAPI

In [ ]:
!pip install retina-face

In [7]:
from tensorflow.keras.models import load_model
import telebot
from telebot import types
import numpy as np
import cv2
from retinaface import RetinaFace

In [ ]:
model=load_model('/content/drive/MyDrive/prsianrec.h5')
bot=telebot.TeleBot("Token")
# btns=telebot.types.ReplyKeyboardMarkup(row_width=1)
# btn1=telebot.types.KeyboardButton('Start')
# btns.add(btn1)
@bot.message_handler(commands=['start'])
def wlc(message):
    bot.send_message(message.chat.id,'Hi 👋🏻 please send me a picture👀',
    #reply_markup=btns
     )

@bot.message_handler(content_types=['photo'])
def photo(message):
    fileID = message.photo[-1].file_id
    file_info = bot.get_file(fileID)
    src=file_info.file_path
    downloaded_file = bot.download_file(src)
    with open(f"/content/drive/MyDrive/{src}", 'wb') as new_file:
        new_file.write(downloaded_file)


    faces = RetinaFace.extract_faces(img_path =f"/content/drive/MyDrive/{src}", align = True)
    for face in faces:
      img_RGB = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
      img_resize = cv2.resize(img_RGB, (224, 224))
      img_numpy = np.array(img_resize)
      img = img_numpy / 255.0
      final = img.reshape(1, 224, 224, 3)
      y_pred = np.argmax(model.predict(final))

    if y_pred == 0:
        bot.reply_to(message,'Foreign')
      
    else:
        bot.reply_to(message,'Persian')
      

bot.polling()